In [16]:
import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Merge, Embedding, Convolution1D,Dropout
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence


from features import *
from helper import *

In [2]:
code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [3]:
import pickle
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [4]:
text = []

for ses_mod in data2:
    text.append(ses_mod['transcription'])

In [5]:
MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
x_train_text = []

x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

In [6]:
import codecs
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

file_loc = data_path + '../glove.42B.300d.txt'

print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 2736 unique tokens
/home/samarth/emotion_recognition-master/code/../data/sessions/../glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 90


In [13]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4936, 4)

In [23]:
model = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model.add(Convolution1D(256, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu')) 
model.add(Dropout(0.2))
model.add(Dense(4))
model.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model.summary()

print("Model1 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 300)          821100    
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 500, 256)          230656    
_________________________________________________________________
dropout_13 (Dropout)         (None, 500, 256)          0         
_________________________________________________________________
activation_13 (Activation)   (None, 500, 256)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 500, 128)          98432     
_________________________________________________________________
dropout_14 (Dropout)         (None, 500, 128)          0         
_________________________________________________________________
activation_14 (Activation)   (None, 500, 128)          0         
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`


In [24]:
hist = model.fit(x_train_text, Y, 
                 batch_size=100, nb_epoch=125, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/125
3948/3948 [==============================] - 5s - loss: 1.3701 - acc: 0.3379 - val_loss: 1.3694 - val_acc: 0.3694
Epoch 2/125
3948/3948 [==============================] - 4s - loss: 1.3621 - acc: 0.3432 - val_loss: 1.3285 - val_acc: 0.3887
Epoch 3/125
3948/3948 [==============================] - 4s - loss: 1.3026 - acc: 0.3744 - val_loss: 1.2241 - val_acc: 0.4494
Epoch 4/125
3948/3948 [==============================] - 4s - loss: 1.2280 - acc: 0.4240 - val_loss: 1.2129 - val_acc: 0.4514
Epoch 5/125
3948/3948 [==============================] - 4s - loss: 1.1324 - acc: 0.4828 - val_loss: 1.2270 - val_acc: 0.4555
Epoch 6/125
3948/3948 [==============================] - 4s - loss: 1.0153 - acc: 0.5415 - val_loss: 1.2696 - val_acc: 0.4727
Epoch 7/125
3948/3948 [==============================] - 4s - loss: 0.9095 - acc: 0.6125 - val_loss: 1.3892 - val_acc: 0.4828
Epoch 8/125
3948/3948 [==============================] - 4s - loss: 0.8

In [25]:
model = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model.add(LSTM(512, return_sequences=True))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))


#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model.summary()

print("Model1 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 300)          821100    
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 512)          1665024   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_7 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_19 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_20 (Activation)   (None, 4)                 0         
Total para

In [26]:
hist = model.fit(x_train_text, Y, 
                 batch_size=100, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 79s - loss: 1.3064 - acc: 0.3934 - val_loss: 1.1940 - val_acc: 0.4808
Epoch 2/30
3948/3948 [==============================] - 80s - loss: 1.0557 - acc: 0.5537 - val_loss: 0.9880 - val_acc: 0.5941
Epoch 3/30
3948/3948 [==============================] - 80s - loss: 0.8372 - acc: 0.6593 - val_loss: 0.9241 - val_acc: 0.6397
Epoch 4/30
3948/3948 [==============================] - 80s - loss: 0.6876 - acc: 0.7305 - val_loss: 0.9942 - val_acc: 0.6063
Epoch 5/30
3948/3948 [==============================] - 79s - loss: 0.5866 - acc: 0.7687 - val_loss: 1.0076 - val_acc: 0.6245
Epoch 6/30
3948/3948 [==============================] - 80s - loss: 0.4834 - acc: 0.8116 - val_loss: 1.1072 - val_acc: 0.6336
Epoch 7/30
3948/3948 [==============================] - 80s - loss: 0.4247 - acc: 0.8293 - val_loss: 1.1809 - val_acc: 0.6306
Epoch 8/30
3948/3948 [==============================] - 80s - loss: 0.3

In [28]:
len(tokenizer.word_index)

2736

In [29]:
model = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Embedding(2736,
                    128))

model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))


#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model.summary()

print("Model1 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 128)         350208    
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 256)         394240    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_9 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_21 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 2052      
_________________________________________________________________
activation_22 (Activation)   (None, 4)                 0         
Total para

In [30]:
hist = model.fit(x_train_text, Y, 
                 batch_size=100, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 65s - loss: 1.3567 - acc: 0.3478 - val_loss: 1.2947 - val_acc: 0.4302
Epoch 2/30
3948/3948 [==============================] - 64s - loss: 1.0827 - acc: 0.5466 - val_loss: 1.0113 - val_acc: 0.5850
Epoch 3/30
3948/3948 [==============================] - 63s - loss: 0.7808 - acc: 0.6920 - val_loss: 1.0052 - val_acc: 0.5992
Epoch 4/30
3948/3948 [==============================] - 62s - loss: 0.6202 - acc: 0.7604 - val_loss: 0.9847 - val_acc: 0.6478
Epoch 5/30
3948/3948 [==============================] - 62s - loss: 0.5411 - acc: 0.7943 - val_loss: 1.0931 - val_acc: 0.6407
Epoch 6/30
3948/3948 [==============================] - 62s - loss: 0.4679 - acc: 0.8207 - val_loss: 1.1445 - val_acc: 0.6134
Epoch 7/30
3948/3948 [==============================] - 64s - loss: 0.4165 - acc: 0.8425 - val_loss: 1.2533 - val_acc: 0.6174
Epoch 8/30
3948/3948 [==============================] - 64s - loss: 0.4